# 5-Day AI Agents Intensive: Assignment 1
## Introduction to AI Agents and Multi-Agent Architectures

---

**Course Overview**

This assignment combines theoretical understanding with practical implementation of AI agents using Google's Agent Development Kit (ADK). You'll learn:

- ✅ Core concepts of AI agents and agentic systems
- ✅ Building single agents with tools
- ✅ Designing multi-agent architectures
- ✅ Implementing workflow patterns (Sequential, Parallel, Loop)

---

## Part 1: Theoretical Foundation

### 1.1 From Predictive AI to Autonomous Agents

Artificial intelligence is undergoing a paradigm shift. Traditional AI models excel at passive, discrete tasks like answering questions or generating images, but they require constant human direction. We're now moving toward **AI agents** - complete applications capable of autonomous problem-solving and task execution.

**Key Distinction:**
- **Traditional AI**: `Prompt → LLM → Text`
- **AI Agent**: `Prompt → Agent → Think → Act → Observe → Final Answer`

### 1.2 What is an AI Agent?

An AI agent combines four essential components:

1. **The Model (The "Brain")**: Core reasoning engine (LM/foundation model) that processes information and makes decisions
2. **Tools (The "Hands")**: Mechanisms connecting reasoning to the real world (APIs, functions, databases)
3. **Orchestration Layer (The "Nervous System")**: Governs the operational loop, manages planning, memory, and reasoning strategies
4. **Deployment (The "Body and Legs")**: Production infrastructure making the agent accessible and reliable

### 1.3 The Agentic Problem-Solving Process

Agents operate on a **five-step cyclical process**:

1. **Get the Mission**: Receive a specific, high-level goal
2. **Scan the Scene**: Gather context from available resources
3. **Think It Through**: Devise a plan using the reasoning model
4. **Take Action**: Execute the plan using appropriate tools
5. **Observe and Iterate**: Analyze outcomes and repeat until goal is achieved

**Example: Customer Support Agent**

User asks: "Where is my order #12345?"

The agent:
1. Plans a multi-step strategy (find order → track shipment → report)
2. Calls `find_order("12345")` tool
3. Observes: order found with tracking number "ZYX987"
4. Calls `get_shipping_status("ZYX987")` tool
5. Observes: "Out for Delivery"
6. Generates response: "Your order #12345 is 'Out for Delivery'!"

### 1.4 Taxonomy of Agentic Systems

**Level 0: Core Reasoning System**
- LM operates in isolation with pre-trained knowledge
- No tools, memory, or real-time data access
- Can explain concepts but cannot access current information

**Level 1: Connected Problem-Solver**
- Connects to external tools (search APIs, databases)
- Can retrieve real-time information
- Example: Searching for current stock prices or weather

**Level 2: Strategic Problem-Solver**
- Plans complex, multi-step tasks
- Employs context engineering to manage information
- Example: Finding coffee shops midway between two locations

**Level 3: Collaborative Multi-Agent System**
- Team of specialized agents working together
- Agents treat other agents as tools
- Example: Project manager delegating to research, marketing, and web dev agents

**Level 4: Self-Evolving System**
- Identifies capability gaps and creates new tools/agents
- Dynamically expands its own capabilities
- Represents the frontier of autonomous systems

### 1.5 Multi-Agent Design Patterns

When tasks grow complex, specialized agent teams become more effective than single "super-agents":

**Coordinator Pattern**: Manager agent routes sub-tasks to specialists

**Sequential Pattern**: Linear workflow where output of one agent becomes input for next

**Iterative Refinement Pattern**: Generator creates content, critic evaluates, loop continues

**Human-in-the-Loop (HITL) Pattern**: Agent pauses for human approval at critical decisions

---

## Part 2: Practical Implementation

### Setup and Configuration

We'll use Google's Agent Development Kit (ADK) to build our agents. ADK provides:
- Modular framework for agent development
- Optimized for Gemini models
- Support for Python, Java, and Go
- Built-in tools and orchestration capabilities

---

### ⚙️ Section 1: Environment Setup

In [5]:
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

In [6]:
# !pip install google-generativeai 
# !pip install google-adk kaggle nbformat 
# !pip install jupyter_server

In [7]:
# import warnings
# warnings.filterwarnings("ignore")

#### 1.1 Configure Gemini API Key

**Instructions:**
1. Get your API key from [Google AI Studio](https://aistudio.google.com/app/api-keys)
2. In Kaggle: Add-ons → Secrets → Create `GOOGLE_API_KEY`
3. Ensure the checkbox is selected to attach the secret

In [12]:
# import os
# from kaggle_secrets import UserSecretsClient

# try:
#     GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
#     os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
#     print("✅ Gemini API key setup complete.")
# except Exception as e:
#     print(f"🔒 Authentication Error: {e}")
#     print("Please add 'GOOGLE_API_KEY' to your Kaggle secrets.")

import os

# 🔑 Replace with your actual Gemini API key from Google AI Studio
os.environ["GOOGLE_API_KEY"] = "......qCI"   #YOUR_API_KEY_HERE

if "GOOGLE_API_KEY" in os.environ:
    print("✅ Gemini API key setup complete.")
else:
    print("❌ API key not found.")


✅ Gemini API key setup complete.


#### 1.2 Import ADK Components

Now, import the specific components you'll need from the Agent Development Kit and the Generative AI library. This keeps your code organized and ensures we have access to the necessary building blocks.

In [15]:
from google.adk.agents import Agent, SequentialAgent, ParallelAgent, LoopAgent
from google.adk.models.google_llm import Gemini
from google.adk.runners import InMemoryRunner
from google.adk.tools import AgentTool, FunctionTool, google_search
from google.genai import types

print("✅ ADK components imported successfully.")

MCP requires Python 3.10 or above. Please upgrade your Python version in order to use it.
MCP requires Python 3.10 or above. Please upgrade your Python version in order to use it.


✅ ADK components imported successfully.


#### Helper functions

We'll define some helper functions. If you are running this outside the Kaggle environment, you don't need to do this.

In [17]:
# Define helper functions that will be reused throughout the notebook

from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

C:\Users\pawar\AppData\Local\Temp\ipykernel_9908\3275386924.py:3: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


✅ Helper functions defined.


#### 🧩 Purpose Summary

This code defines a helper function get_adk_proxy_url() that helps connect to and open the Google ADK (Agent Development Kit) Web UI inside a Kaggle Notebook environment.

#### 1.3 Configure Retry Options

Handle transient errors like rate limits with exponential backoff:

In [20]:
retry_config = types.HttpRetryOptions(
    attempts=5,  # Maximum retry attempts
    exp_base=7,  # Delay multiplier
    initial_delay=1,  # Initial delay before first retry (seconds)
    http_status_codes=[429, 500, 503, 504]  # Retry on these HTTP errors
)

print("✅ Retry configuration set.")

✅ Retry configuration set.


---

## Task 1: Building Your First AI Agent

### Objective
Create a single agent that can search for current information and provide informed responses.

### What You'll Learn
- How to configure an agent with a model, instructions, and tools
- How agents use tools to access real-time information
- The difference between static LM responses and agent-powered responses

### 🤔 What is an AI Agent?

You've probably used an LLM like Gemini before, where you give it a prompt and it gives you a text response.

`Prompt -> LLM -> Text`

An AI Agent takes this one step further. An agent can think, take actions, and observe the results of those actions to give you a better answer.

`Prompt -> Agent -> Thought -> Action -> Observation -> Final Answer`

---

### Step 1: Define Your First Agent

We'll create an agent with:
- **Model**: Gemini 2.5 Flash Lite (fast, efficient)
- **Instruction**: Clear guidance on when to use tools
- **Tools**: Google Search for current information

These are the main properties we'll set:

- **name** and **description**: A simple name and description to identify our agent.
- **model**: The specific LLM that will power the agent's reasoning. We'll use "gemini-2.5-flash-lite".
- **instruction**: The agent's guiding prompt. This tells the agent what its goal is and how to behave.
- **tools**: A list of [tools](https://google.github.io/adk-docs/tools/) that the agent can use. To start, we'll give it the `google_search` tool, which lets it find up-to-date information online.

In [23]:
# Create a helpful assistant agent with Google Search capability
root_agent = Agent(
    name="helpful_assistant",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    description="A simple agent that can answer general questions.",
    instruction="You are a helpful assistant. Use Google Search for current info or if unsure.",
    tools=[google_search],
)

print("✅ Root Agent defined.")

✅ Root Agent defined.


### Step 2: Run Your Agent

Create a runner and send a query that requires current information:

In [25]:
# Create a runner for the agent
runner = InMemoryRunner(agent=root_agent)
print("✅ Runner created.")

# Ask about ADK - requires current information
response = await runner.run_debug(
    "What is Agent Development Kit from Google? What languages is the SDK available in?"
)

✅ Runner created.

 ### Created new session: debug_session_id

User > What is Agent Development Kit from Google? What languages is the SDK available in?
helpful_assistant > The Agent Development Kit (ADK) from Google is a flexible and modular framework designed to simplify the development, deployment, and orchestration of AI agents and multi-agent systems. It applies software development principles to AI agent creation, aiming to make building complex agentic architectures feel more like traditional software development. While optimized for Google's Gemini models and ecosystem, ADK is model-agnostic and can be used with other frameworks.

The ADK allows developers to define agent logic, tools, and orchestration directly in code, offering benefits such as robust debugging, reliable versioning, and deployment flexibility. It supports a rich tool ecosystem, including pre-built tools, custom functions, and third-party integrations. ADK also facilitates the creation of modular multi-agent s

### Step 3: Try Your Own Query

Test the agent with a question requiring current information:

In [27]:
# Try asking about current events or information
response = await runner.run_debug("What's the weather in London?")

# Feel free to modify the query above and try:
# - "Who won the last soccer world cup?"
# - "What new movies are showing in theaters now?"
# - Your own question!


 ### Continue session: debug_session_id

User > What's the weather in London?
helpful_assistant > The weather in London is currently cloudy with a temperature of 58°F (14°C). There is a 0% chance of rain today. The humidity is around 79%.

The forecast for the next few days includes a chance of light rain and clouds. Temperatures are expected to range from the low 50s°F to the low 60s°F (around 11°C to 16°C) over the next week.


### 🎯 Task 1 Reflection

**Key Takeaways:**
1. The agent didn't just respond - it **reasoned** that it needed more information
2. It **acted** by using the Google Search tool
3. It **observed** the results and formulated an answer
4. This ability to take action is the foundation of agent-based AI

**What's Next?**
A single agent is powerful, but complex tasks require teams. Let's build multi-agent systems!

---

## 💻 Section 3: Try the ADK Web Interface

### Overview

ADK includes a built-in web interface for interactively chatting with, testing, and debugging your agents.

<img width="1200" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/adk-web-ui.gif" alt="ADK Web UI" />

To use the ADK web UI, you'll need to create an agent with Python files using the `adk create` command.

Run the command below to generate a `sample-agent` folder that contains all the necessary files, including `agent.py` for your code, an `.env` file with your API key pre-configured, and an `__init__.py` file:

## Task 2: Multi-Agent Systems & Workflow Patterns

### Objective
Build specialized agent teams that collaborate using different workflow patterns.

### The Problem with Monolithic Agents
Single "do-it-all" agents become:
- Hard to debug (which part failed?)
- Difficult to maintain (long, complex instructions)
- Unreliable (trying to do too much)

### The Solution: Team of Specialists
Multiple simple agents, each with one clear job, collaborating like a real team.


**Architecture: Single Agent vs Multi-Agent Team**


<img width="800" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/multi-agent-team.png" alt="Multi-agent Team" />

### Pattern 1: LLM-Based Orchestration

Let an LLM coordinator decide which specialist agents to call and when.

**Example: Research & Summarization System**


1. **Research Agent** - Searches for information using Google Search
2. **Summarizer Agent** - Creates concise summaries from research findings

In [33]:
# Research Agent: Searches for information
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic 
    and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",
)

print("✅ research_agent created.")

✅ research_agent created.


In [34]:

# Summarizer Agent: Creates concise summaries
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

print("✅ summarizer_agent created.")

✅ summarizer_agent created.


In [35]:
# Root Coordinator: Orchestrates the workflow
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information.
2. Next, after receiving findings, you MUST call the `SummarizerAgent` tool.
3. Finally, present the final summary clearly to the user.""",
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ root_agent created.")

✅ root_agent created.


In [36]:
# Research Agent: Searches for information
research_agent = Agent(
    name="ResearchAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a specialized research agent. Your only job is to use the
    google_search tool to find 2-3 pieces of relevant information on the given topic 
    and present the findings with citations.""",
    tools=[google_search],
    output_key="research_findings",
)

# Summarizer Agent: Creates concise summaries
summarizer_agent = Agent(
    name="SummarizerAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Read the provided research findings: {research_findings}
Create a concise summary as a bulleted list with 3-5 key points.""",
    output_key="final_summary",
)

# Root Coordinator: Orchestrates the workflow
root_agent = Agent(
    name="ResearchCoordinator",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""You are a research coordinator. Your goal is to answer the user's query by orchestrating a workflow.
1. First, you MUST call the `ResearchAgent` tool to find relevant information.
2. Next, after receiving findings, you MUST call the `SummarizerAgent` tool.
3. Finally, present the final summary clearly to the user.""",
    tools=[AgentTool(research_agent), AgentTool(summarizer_agent)],
)

print("✅ LLM-orchestrated multi-agent system created.")

✅ LLM-orchestrated multi-agent system created.


In [37]:
# Run the multi-agent system
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "What are the latest advancements in quantum computing and what do they mean for AI?"
)


 ### Created new session: debug_session_id

User > What are the latest advancements in quantum computing and what do they mean for AI?


ResearchCoordinator > Quantum AI is poised to revolutionize artificial intelligence by leveraging the immense computational power of quantum computers. Key advancements include:

*   **Enhanced Computational Power & Speed:** Quantum computers can process vast amounts of data and explore numerous solutions simultaneously, drastically accelerating AI tasks like model training and complex simulations. Some quantum algorithms have demonstrated performance thousands of times faster than classical supercomputers.
*   **Improved AI Capabilities & Optimization:** Quantum machine learning algorithms promise more efficient data processing for better pattern recognition and natural language processing. Additionally, quantum algorithms are expected to significantly enhance AI's optimization capabilities, crucial for complex problems in fields like supply chain management and financial modeling.
*   **Sustainability & Hybrid Approaches:** Quantum AI models may require fewer parameters, potentially 

### Pattern 2: Sequential Workflow - The Assembly Line

**Use Case**: When tasks must happen in a specific, guaranteed order.

**Example: Blog Post Creation Pipeline**

<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/sequential-agent.png" alt="Sequential Agent" />

#### Example: Blog Post Creation with Sequential Agents

Let's build a system with three specialized agents:

1. **Outline Agent** - Creates a blog outline for a given topic
2. **Writer Agent** - Writes a blog post
3. **Editor Agent** - Edits a blog post draft for clarity and structure

In [40]:
# Outline Agent: Creates blog structure
outline_agent = Agent(
    name="OutlineAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Create a blog outline for the given topic with:
    1. A catchy headline
    2. An introduction hook
    3. 3-5 main sections with 2-3 bullet points for each
    4. A concluding thought""",
    output_key="blog_outline",
)

# Writer Agent: Writes full blog post
writer_agent = Agent(
    name="WriterAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Following this outline strictly: {blog_outline}
    Write a brief, 200 to 300-word blog post with an engaging and informative tone.""",
    output_key="blog_draft",
)

# Editor Agent: Polishes the draft
editor_agent = Agent(
    name="EditorAgent",
    model=Gemini(
        model="gemini-2.5-flash-lite",
        retry_options=retry_config
    ),
    instruction="""Edit this draft: {blog_draft}
    Polish by fixing grammatical errors, improving flow and sentence structure, 
    and enhancing overall clarity.""",
    output_key="final_blog",
)

# Sequential Agent: Runs agents in fixed order
root_agent = SequentialAgent(
    name="BlogPipeline",
    sub_agents=[outline_agent, writer_agent, editor_agent],
)

print("✅ Sequential Agent created.")

✅ Sequential Agent created.


In [41]:
# Run the sequential pipeline
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a blog post about the benefits of multi-agent systems for software developers"
)


 ### Created new session: debug_session_id

User > Write a blog post about the benefits of multi-agent systems for software developers
OutlineAgent > ## Blog Outline:

**Headline:** Unleash the Power of Collaboration: How Multi-Agent Systems Will Revolutionize Your Development Workflow

**Introduction Hook:** Imagine a world where your code writes itself, complex bugs are automatically squashed, and your development team operates with unprecedented efficiency. This isn't science fiction; it's the promise of Multi-Agent Systems (MAS). For software developers, understanding and leveraging MAS is no longer a niche interest but a pathway to a more powerful, agile, and innovative future.

**Main Sections:**

**1. What Exactly is a Multi-Agent System? (And Why Should You Care?)**
    *   **Definition:** Briefly explain MAS as a system composed of multiple autonomous agents that interact with each other and their environment to achieve individual and collective goals.
    *   **The "Develope

### Pattern 3: Parallel Workflow - Independent Researchers

**Use Case**: When tasks are independent and can run simultaneously for speed.

**The Solution: Concurrent Execution**

When you have independent tasks, you can run them all at the same time using a `ParallelAgent`. This agent executes all of its sub-agents concurrently, dramatically speeding up the workflow. Once all parallel tasks are complete, you can then pass their combined results to a final 'aggregator' step.

**Use Parallel when:** Tasks are independent, speed matters, and you can execute concurrently.

**Example: Multi-Topic Research**

<img width="600" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/parallel-agent.png" alt="Parallel Agent" />

#### Example: Parallel Multi-Topic Research

Let's build a system with four agents:

1. **Tech Researcher** - Researches AI/ML news and trends
2. **Health Researcher** - Researches recent medical news and trends
3. **Finance Researcher** - Researches finance and fintech news and trends
4. **Aggregator Agent** - Combines all research findings into a single summary

In [44]:
# Create three specialized researchers
tech_researcher = Agent(
    name="TechResearcher",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Research the latest AI/ML trends. Include 3 key developments,
the main companies involved, and the potential impact. Keep the report very concise (100 words).""",
    tools=[google_search],
    output_key="tech_research",
)

health_researcher = Agent(
    name="HealthResearcher",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Research recent medical breakthroughs. Include 3 significant advances,
their practical applications, and estimated timelines. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="health_research",
)

finance_researcher = Agent(
    name="FinanceResearcher",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Research current fintech trends. Include 3 key trends,
their market implications, and the future outlook. Keep the report concise (100 words).""",
    tools=[google_search],
    output_key="finance_research",
)

# Aggregator combines all research
aggregator_agent = Agent(
    name="AggregatorAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Combine these three research findings into a single executive summary:

 **Technology Trends:** {tech_research}
 **Health Breakthroughs:** {health_research}
 **Finance Innovations:** {finance_research}
 
 Highlight common themes, surprising connections, and key takeaways. 
 Final summary should be around 200 words.""",
    output_key="executive_summary",
)

# Parallel execution then sequential aggregation
parallel_research_team = ParallelAgent(
    name="ParallelResearchTeam",
    sub_agents=[tech_researcher, health_researcher, finance_researcher],
)

root_agent = SequentialAgent(
    name="ResearchSystem",
    sub_agents=[parallel_research_team, aggregator_agent],
)

print("✅ Parallel and Sequential Agents created.")

✅ Parallel and Sequential Agents created.


In [46]:
# Run parallel research with aggregation
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Run the daily executive briefing on Tech, Health, and Finance"
)


 ### Created new session: debug_session_id

User > Run the daily executive briefing on Tech, Health, and Finance
HealthResearcher > Here are three recent medical breakthroughs:

1.  **AI-driven Sepsis Detection:** Artificial intelligence algorithms can now detect sepsis nearly six hours earlier than traditional methods.
    *   **Application:** This allows for faster treatment, significantly reducing sepsis-related deaths and hospitalizations. It's also being adapted to predict kidney and heart failure.
    *   **Timeline:** Already being implemented in hospitals, with widespread adoption expected within the next 1-2 years.

2.  **Gene Sequencing for Rare Diseases:** Rapid gene sequencing is identifying dangerous mutations, aiding in the diagnosis of rare genetic disorders.
    *   **Application:** Enables earlier and more accurate diagnoses, leading to targeted therapies and improved patient outcomes.
    *   **Timeline:** Advanced gene sequencing is increasingly accessible, with sig

### Pattern 4: Loop Workflow - The Refinement Cycle

**Use Case**: When output needs iterative improvement through feedback cycles.

**Example: Story Writing & Critique Loop**

<img width="250" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/loop-agent.png" alt="Loop Agent" />

In [49]:
# Initial Writer: Creates first draft
initial_writer_agent = Agent(
    name="InitialWriterAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""Based on the user's prompt, write the first draft of a short story 
    (around 100-150 words). Output only the story text, with no introduction or explanation.""",
    output_key="current_story",
)

# Critic: Provides feedback or approval
critic_agent = Agent(
    name="CriticAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a constructive story critic. Review the story: {current_story}
    
    Evaluate plot, characters, and pacing.
    - If the story is well-written and complete, respond EXACTLY: "APPROVED"
    - Otherwise, provide 2-3 specific, actionable suggestions for improvement.""",
    output_key="critique",
)

# Exit function for loop termination
def exit_loop():
    """Call this function ONLY when the critique is 'APPROVED', indicating 
    the story is finished and no more changes are needed."""
    return {"status": "approved", "message": "Story approved. Exiting refinement loop."}

# Refiner: Improves story or exits loop
refiner_agent = Agent(
    name="RefinerAgent",
    model=Gemini(model="gemini-2.5-flash-lite", retry_options=retry_config),
    instruction="""You are a story refiner. You have:
    Story Draft: {current_story}
    Critique: {critique}
    
    Analyze the critique:
    - IF critique is EXACTLY "APPROVED", call the `exit_loop` function and nothing else.
    - OTHERWISE, rewrite the story to fully incorporate the feedback.""",
    output_key="current_story",
    tools=[FunctionTool(exit_loop)],
)

# Loop for refinement cycles
story_refinement_loop = LoopAgent(
    name="StoryRefinementLoop",
    sub_agents=[critic_agent, refiner_agent],
    max_iterations=2,  # Prevents infinite loops
)

# Root Sequential Agent: Initial Write → Refinement Loop
root_agent = SequentialAgent(
    name="StoryPipeline",
    sub_agents=[initial_writer_agent, story_refinement_loop],
)

print("✅ Loop and Sequential Agents created.")

✅ Loop and Sequential Agents created.


In [51]:
# Run the story refinement system
runner = InMemoryRunner(agent=root_agent)
response = await runner.run_debug(
    "Write a short story about a lighthouse keeper who discovers a mysterious, glowing map"
)


 ### Created new session: debug_session_id

User > Write a short story about a lighthouse keeper who discovers a mysterious, glowing map
InitialWriterAgent > Elias traced the salt-laced lines on the storm-battered parchment. It wasn't paper, not quite. It felt more like woven moonlight, cool and impossibly smooth beneath his calloused fingertips. He’d found it tucked inside a barnacle-encrusted sea chest that had washed ashore after the last tempest. The map depicted no known coastline, but strange, swirling constellations and a single, pulsating emerald dot at its center. As he held it, the dot flared, projecting a faint, spectral beam onto the lighthouse wall. Elias shivered, a prickle of both fear and exhilaration racing down his spine. The sea, it seemed, still held secrets he'd never dreamed of.
CriticAgent > The story is a compelling start, hinting at a grand adventure. Here are a few suggestions to enhance it:

*   **Character Depth:** While Elias's initial reaction is well-des

### 🎯 Task 2 Reflection: Choosing the Right Pattern

**Decision Tree: Which Workflow Pattern?**

| Pattern | When to Use | Example | Key Feature |
|---------|-------------|---------|-------------|
| **LLM-based Orchestration** | Dynamic decisions needed | Research + Summarize | LLM decides what to call |
| **Sequential** | Order matters, linear pipeline | Outline → Write → Edit | Deterministic order |
| **Parallel** | Independent tasks, speed matters | Multi-topic research | Concurrent execution |
| **Loop** | Iterative improvement needed | Writer + Critic refinement | Repeated cycles |

<img width="1000" src="https://storage.googleapis.com/github-repo/kaggle-5days-ai/day1/agent-decision-tree.png" alt="Agent Decision Tree" />

**Key Takeaways:**
- Multi-agent systems scale better than monolithic agents
- Choose patterns based on task requirements (order, independence, iteration)
- Use `output_key` for state passing between agents
- Always include safeguards like `max_iterations` in loops

---